In [ ]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import glob
import math
from enum import Enum
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
from sklearn.model_selection import KFold
import math
import pickle
import datetime
import os 
from sklearn.model_selection import KFold
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
BATCH_SIZE = 64
CNN_KERNAL_SIZE = 3
#TODO: build a validation/test split
k_folds = 5
num_epochs = 1

In [ ]:
#Check if GPU is available and select if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
device = torch.device("cuda")

In [ ]:
#importing user data

subjects = []
for i in range (1,31):
    interimPysch = pd.read_csv('.\\CASE_full\\data\\non-interpolated\\physiological\\sub_' + str(i) + '.csv')
    interimPysch = interimPysch.drop(columns=['video'])
    interimPysch['daqtime'] = interimPysch['daqtime'].astype('int32')
    interimAnote = pd.read_csv('.\\CASE_full\\data\\non-interpolated\\annotations\\sub_' + str(i) + '.csv')
    interimAnote = interimAnote.drop(columns=['video'])
    interimAnote['jstime'] = interimAnote['jstime'].astype('int32')
    final = interimPysch.merge(interimAnote, left_on='daqtime',right_on='jstime', how='outer')
    #jstime dropped as it is redundant
    final = final.drop(columns=['jstime','daqtime'])
    final = final.ffill()
    final = final.bfill()
    final.to_csv('.\\Processed-Data\\KfoldProcess\\sub_' + str(i) + '_processed'+'.csv')
    subjects.append(final)
subjects[0]



In [ ]:

def createStressLevel(Dataframe):
    vals = []
    valence = Dataframe.loc[:,'valence']
    arousal = Dataframe.loc[:,'arousal']
    for i in range(0,len(valence)):
        vals.append(round((arousal[i]-0.5)*math.cos((math.pi/18)*(valence[i] - 0.5))))
    return vals


In [ ]:
#for KFOLD we load everything into one dataset and then split later down the road.
subjects = []

for i in range(1,31):
      temp = pd.read_csv('.\\Processed-Data\\Filled-Data\\sub_' + str(i) + '_processed'+'.csv')
      temp['StressLevel'] = createStressLevel(temp)
      subjects.append(temp)


   

In [ ]:

subjects[1]

In [ ]:
#combining Dataset and train set frames
#it may be required that timestamps be dropped from the prediciton model entirely, or atleast switched to date time, as the model
#may try to learn emotions based on when measuring starts and finishes. This could be problematic
#as sessions in the field could go longer than eny training sessions, or disconnects could restart sessions
datasetFrame = pd.concat(subjects.copy())
datasetFrame = datasetFrame.drop(columns=['Unnamed: 0'])
Labels = torch.Tensor(datasetFrame['StressLevel'].values.astype(int))
datasetFrame = datasetFrame.drop(columns=['StressLevel','valence','arousal'])

datasetFrame.to_csv('.\\Processed-Data\\KFOLD-FULL\\trainingData.csv',index=False)
pickle.dump(Labels, open('.\\Processed-Data\\KFOLD-FULL\\Labels.pkl','wb'))

  


In [ ]:
Labels = pickle.load(open('.\\Processed-Data\\KFOLD-FULL\\Labels.pkl','rb')).to(device)

datasetFrame = pd.read_csv('.\\Processed-Data\\KFOLD-FULL\\trainingData.csv')


In [ ]:
trainTensor = torch.tensor(datasetFrame.values.astype(float),dtype=torch.float32).to(device)
print(f"Shape: {trainTensor.shape} Type: {trainTensor.dtype}")


In [ ]:
Data_Set = TensorDataset(trainTensor,Labels)
print(f'Data_Set Length: {len(Data_Set)}')

In [ ]:
def reset_weights(model):
  """performed to avoid weight leakage"""
  for layer in model.children():
    if hasattr(layer, 'reset_parameters'):
      layer.reset_parameters()

In [ ]:
#definining the model
#proposed function for stress level S = A/V where S is stress, A is arousal, and V is valence
#since valence represents the positivity of the emotion, it would be inversely proportional to stress as higher valence means a better emotion
#Since higher arousal can be generally translated to 
class StressScanner(torch.nn.Module):
    def __init__(self):
        super(StressScanner, self).__init__()
        self.ConvolutionIn = torch.nn.Conv1d(in_channels=8, out_channels=16,kernel_size=CNN_KERNAL_SIZE,padding=1)
        self.ConvHidden1 = torch.nn.Conv1d(in_channels=16, out_channels=32,kernel_size=CNN_KERNAL_SIZE,padding=1)
        self.ConvHidden2 = torch.nn.Conv1d(in_channels=32, out_channels=32,kernel_size=CNN_KERNAL_SIZE,padding=1)
        self.ConvHidden3 = torch.nn.Conv1d(in_channels=32, out_channels=20,kernel_size=CNN_KERNAL_SIZE,padding=1)
        self.ConvOUT = torch.nn.Conv1d(in_channels=20, out_channels=10,kernel_size=CNN_KERNAL_SIZE,padding=1)

    def forward(self, x):
        x = F.relu(self.ConvolutionIn(x))
        x = F.relu(self.ConvHidden1(x))
        x = F.relu(self.ConvHidden2(x))
        x = F.relu(self.ConvHidden3(x))
        x = self.ConvOUT(x)
        return x

In [ ]:
torch.manual_seed(123)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss().to(device)
kfold = KFold(n_splits=k_folds, shuffle=True)
results = {}

In [ ]:
#Train loop
#Note: tb_writer is just a tensorboard writer for statistics
def train():
    current_loss = 0.0
    last_loss = 0.0
    for fold , (train_id, test_id) in enumerate(kfold.split(Data_Set)):
        print("FOLD: ", fold)

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_id)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_id)
        #Preparing training and testing data using subsampling
        Train_Loader = DataLoader(Data_Set,batch_size=BATCH_SIZE,sampler=train_subsampler)
        Test_Loader = DataLoader(Data_Set,batch_size=BATCH_SIZE,sampler=test_subsampler)
        
        #initializing the model
        model = StressScanner()
        model.apply(reset_weights)
        model = model.to(device)
        #initialize optimizer
        optimizer = torch.optim.ASGD(model.parameters(), lr=0.0001)
        
        #training function
        for epoch in range (0,num_epochs):
            print(f'Epoch: {epoch}')
            current_loss = 0.0
            
            for i, batch in enumerate(Train_Loader, 0):
                input, labels = batch
                optimizer.zero_grad()
                
                input = input.unsqueeze(2)
                output = model(input)
                
                labels = labels.long()
                labels = labels.unsqueeze(1)
                loss = loss_fn(output,labels)
                loss.backward()
                
                optimizer.step()
                
                current_loss += loss.item() 
                if (i % 1000 == 0):
                    print(f'Epoch: {epoch} Loss: {current_loss}')
                    last_loss = current_loss
                    current_loss = 0.0
            print("A model has been trained. Saving it now...")
            torch.save(model.state_dict(), f"model_{fold}.pth")
            print("Testing model now...")
        with torch.no_grad():
            total = 0
            correct = 0
            for i, batch in enumerate(Test_Loader, 0):
                

                input, labels = batch
                input = input.unsqueeze(2)
                output = model(input)
                output = output.unsqueeze(2)

                #get predicted values from the model
                _, predicted = torch.max(output.data,1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                results[fold] = (correct/total)*100
                print(f"{fold} Fold Accuracy: {results[fold]}")
        print("CROSS VALIDATION COMPLETE")
        print(f"TOTAL FOLDS: {k_folds}")
        print(f"AVERAGE ACCURACY: {sum(results.values())/total}")
    return last_loss

In [ ]:
train()